## Architectural Overview/Design
![alt text](https://raw.githubusercontent.com/alvi2496/DeCaf/master/assets/word_embedding.png)

## Objective
The main objective is to classify discussions from pull request, issue tracker commit messages and code comments as `design` or `general`. We also want to make the classifier cross project compatible.

## Data Collection
- We intent to have three types of data. One data is to train the `Word Embedding` model. As `Word Embedding` requires structured form of literature, we have used sentences from literatures(ex. papers and books). Also we have restricted our choice of papers and books to only from the Software Engineering domain for keep the context of our `Word Embedding` model restricted to Software Engineering. Our `Word Embedding` model will be used to vectorize our train data.

## Data Cleaning
Raw data can have a lot of noise. Specially when scraped from documents of website, it can contain a lot of misinformation in the form of names, punctuations, numbers(ex. years), misspelled and incompleted words. Also it can have a lot of stopwords that can make the model confused. We have removed all this to make our data as clean as possible. After the cleaning process, out data only contains words that are not stopwords, present in the english dictionary and has lenght greater than three.

In [0]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Assign Data location
- literature: holds the data for word embedding

In [0]:
literature_file = "/content/drive/My Drive/documents/projects/DeCaf/data/literature.txt"
enhanced_literature_file = "/content/drive/My Drive/documents/projects/DeCaf/data/enhanced_literature.txt"
so_model_file = "/content/drive/My Drive/documents/projects/DeCaf/models/pre_trained/so.bin"
we_model_file = "/content/drive/My Drive/documents/projects/DeCaf/models/we.bin"
enhanced_we_model_file = "/content/drive/My Drive/documents/projects/DeCaf/models/enhanced_we.bin"
enhanced_we_model_file_300d = "/content/drive/My Drive/documents/projects/DeCaf/models/enhanced_we_300d.bin"

## Enhance the Literature
- We enhance the `literature` using pre-trained model `so.bin`
  - We are taking every word of literature
  - Inject additional similar word related to software engineering to enhance.

### Read the literature.txt file

In [0]:
literature = open(literature_file, 'r')
words = literature.read().split(" ")
literature.close()
print("Total words in literature: ", len(words)) 

Total words in literature:  1575439


### Divide the words in chunks
- We divide the words in chunks to implement multiprocessing on each chunks
- We take 300000 words in each chunks leading to 5 chunks

In [0]:
file_chunks = []
file_chunks_names = []
index = 0
while index < len(words) - 1:
  chunk = []
  lower_limit = index
  i = 0
  while i < 4 and lower_limit < len(words) - 1:
    upper_limit = lower_limit + 75000
    if upper_limit > len(words):
      upper_limit = len(words) - 1
    chunk.append(words[lower_limit:upper_limit])
    print("lower_limit: " + str(lower_limit) + " upper_limit: " + str(upper_limit))
    lower_limit = upper_limit
    i = i + 1
  file_chunks_names.append(str(index) + '_' + str(lower_limit))
  index = lower_limit
  file_chunks.append(chunk)
print(file_chunks_names)

lower_limit: 0 upper_limit: 75000
lower_limit: 75000 upper_limit: 150000
lower_limit: 150000 upper_limit: 225000
lower_limit: 225000 upper_limit: 300000
lower_limit: 300000 upper_limit: 375000
lower_limit: 375000 upper_limit: 450000
lower_limit: 450000 upper_limit: 525000
lower_limit: 525000 upper_limit: 600000
lower_limit: 600000 upper_limit: 675000
lower_limit: 675000 upper_limit: 750000
lower_limit: 750000 upper_limit: 825000
lower_limit: 825000 upper_limit: 900000
lower_limit: 900000 upper_limit: 975000
lower_limit: 975000 upper_limit: 1050000
lower_limit: 1050000 upper_limit: 1125000
lower_limit: 1125000 upper_limit: 1200000
lower_limit: 1200000 upper_limit: 1275000
lower_limit: 1275000 upper_limit: 1350000
lower_limit: 1350000 upper_limit: 1425000
lower_limit: 1425000 upper_limit: 1500000
lower_limit: 1500000 upper_limit: 1575000
lower_limit: 1575000 upper_limit: 1575438
['0_300000', '300000_600000', '600000_900000', '900000_1200000', '1200000_1500000', '1500000_1575438']


### Import the so word embedding

In [0]:
from gensim.models.keyedvectors import KeyedVectors
import warnings
import os
from datetime import datetime as dt


warnings.simplefilter("ignore")

so_model = KeyedVectors.load_word2vec_format(so_model_file, binary=True)

### Inject similar words

In [0]:
def inject_similar_words(process_number, words, injected_word_chunks):
  start_time = dt.now()
  for i in range(len(words)):
    try:
      words_to_inject = [words[i]]
      similar_words = so_model.most_similar(words[i])
      for similar_tuple in similar_words:
        if similar_tuple[1] > 0.5:
          words_to_inject.append(similar_tuple[0])
      words[i] = " ".join(words_to_inject)
    except:
      continue
    if i % 10000 == 0:
      t = dt.now() - start_time
      start_time = dt.now()
      print("CPU " + str(process_number) + ": Processed: " + str(i) + " / " + str(len(words)) + " words in time: ", t)
  injected_word_chunks[process_number] = words

In [0]:
import multiprocessing as mp

print("Available CPUs: ", mp.cpu_count())
index = 0
for word_chunks in file_chunks:
  literature_chunk_name = "/content/drive/My Drive/documents/projects/DeCaf/data/literature_chunks/" + file_chunks_names[index] + ".txt"
  if not os.path.exists(literature_chunk_name):
    print("Working on chunk: ", file_chunks_names[index])
    manager = mp.Manager()
    injected_word_chunks = manager.dict()
    jobs = []

    for i in range(len(word_chunks)):
      p = mp.Process(target=inject_similar_words, args=(i, word_chunks[i], injected_word_chunks))
      jobs.append(p)
      p.start()

    for proc in jobs:
      proc.join()

    injected_word_chunks = injected_word_chunks.values()

    corpus = []
    for chunk in injected_word_chunks:
      corpus.append(" ".join(chunk))
    corpus = " ".join(corpus)
    literature = open(literature_chunk_name, 'w')
    literature.write(corpus)
    literature.close()
  index = index + 1


Available CPUs:  2


### Concatenate and save the chunks

In [0]:
if not os.path.exists(enhanced_literature_file):
  chunks_directory = "/content/drive/My Drive/documents/projects/DeCaf/data/literature_chunks/"
  print(chunks_names)
  with open(enhanced_literature_file, 'w') as outfile:
    for names in chunks_names:
      with open(chunks_directory + names) as infile:
        outfile.write(infile.read())
      outfile.write(" ")
  print('File created!')

## Create Word Embedding
- Use fasttext for word embedding
  - we take literature.txt as our input data.
  - we train the classifier upsupervised since we just want to group the data according to the similarity.
  - we have used skipgram as we have observe that skipgram models works better with subword information that cbow
  - we are taking words with character number from 4-20. since we are removing every word less than three characters, its not important to take the characters less than 4 characters. Also, the design words seems to be on the bigger side. for ex. reproduceability contains 16 characters. we are considering characters upto 25 characters.
  - We are taking 300 dimension of the words. Also looping for 10 epochs. Both because the training corpus is relatively small.

In [0]:
!pip install fasttext

     |████████████████████████████████| 61kB 2.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2389317 sha256=8377c064a22b0c6932c1a4bb1eb82fdf5f30ae65fa7309f6c16c79420ca4c8af
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


## Create Logging Methods

In [0]:
import pandas as pd

def log(log_file_path, message):
  file = pd.DataFrame([[str(dt.now()), 'info', message]], columns=['timestamp', 'type', 'message'])
  if not os.path.exists(log_file_path):
    file.to_csv(log_file_path)
  else:
    file.to_csv(log_file_path, mode='a', header=False)

def log_result(log_file_path, message):
  f = open(log_file_path, "a")
  f.write(str(dt.now()))
  f.write(message)
  f.close

## Train the `Word Embedding` model and save it as we.bin
- Train and save the model if the model is not present
- Load the model from disk if preselt

In [0]:
import fasttext as ft

log_file_path = "/content/drive/My Drive/documents/projects/DeCaf/logs/we.csv"
we_log = []

if not os.path.exists(enhanced_we_model_file_300d):
  print(str(dt.now())+' Training Word Embedding model...')
  we_log.append(str(dt.now())+' Training Word Embedding model...')
  we_model = ft.train_unsupervised(enhanced_literature_file, "skipgram", minn=4, \
                                   maxn=25, dim=300, epoch=10)
  we_model.save_model(enhanced_we_model_file_300d)
  print(str(dt.now())+' Model trained and saved successfully')
  we_log.append(str(dt.now())+' Model trained and saved successfully')
else:
  print(str(dt.now())+' Loading Word Embedding model from disk...')
  we_log.append(str(dt.now())+' Loading Word Embedding model from disk...')
  we_model = ft.load_model(enhanced_we_model_file_300d)
  print(str(dt.now())+' Model loaded successfully.')
  we_log.append(str(dt.now())+' Model loaded successfully.')

log(log_file_path, \
    "\n".join(we_log))

2020-04-09 02:49:32.534620 Training Word Embedding model...
2020-04-09 08:35:44.445178 Model trained and saved successfully


## Playing around with the Word Embedding model

Get the dimention of the model

In [0]:
we_model.get_dimension()

300

Get the words of the model... For a demo... 

In [0]:
we_model.get_words()

['systems',
 'applications',
 'software',
 'therefore',
 'programs',
 'objects',
 'softwares',
 'avionics',
 'hardware',
 'since',
 'hardware/software',
 'program',
 'sofware',
 's/w',
 'software/hardware',
 'development',
 'application',
 'developers',
 'object',
 'need',
 'uses',
 'otherwise',
 'values',
 'either',
 'means',
 'actually',
 'data',
 'want',
 'defines',
 'architecture',
 'corresponding',
 'architectural',
 'information',
 'entities',
 'system',
 'model',
 'sytems',
 'class',
 'process',
 'multiple',
 'must',
 'designing',
 'always',
 'two',
 'supports',
 'records',
 'classes',
 'table',
 'methodology',
 'techniques',
 'given',
 'use',
 'used',
 'using',
 'tests',
 'collection',
 'defined',
 'approaches',
 'instance',
 'record',
 'will',
 'constructor',
 'database',
 'models',
 'three',
 'testing',
 'processes',
 'service',
 'design',
 'test',
 'associated',
 'strategies',
 'task',
 'approach',
 'fact',
 'environments',
 'computers',
 'customer',
 'method',
 'wish',
 're

Get the vector of a word

In [0]:
we_model.get_word_vector('Maintainability')[0:20]

array([ 0.07752886, -0.19894521, -0.1421423 , -0.32111043, -0.48004603,
       -0.06075421, -0.12335577,  0.12761173, -0.37535232,  0.15787311,
        0.05555912, -0.44184476, -0.39095154, -0.1285222 ,  0.00115345,
        0.2644709 ,  0.22110255,  0.02852138,  0.26432577, -0.33611   ],
      dtype=float32)

Get the vector of a sentence

In [0]:
vector = we_model.get_sentence_vector("Add performance tracker to active admin jobs")
print(vector.shape)

(300,)
